In [1]:
import os
import sys
import time
import networkx as nx
import scipy.io as sio
import scipy.sparse as sp
from scipy.spatial import distance_matrix
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.lines import Line2D
from matplotlib.animation import FuncAnimation

import pickle

%matplotlib inline

import re
numeric_const_pattern = r"""
     [-+]? 
     (?:
         (?: \d* \. \d+ )
         |
         (?: \d+ \.? )
     )
     (?: [Ee] [+-]? \d+ ) ?
     """
rx = re.compile(numeric_const_pattern, re.VERBOSE)

from IPython.display import clear_output
from IPython.display import Video

from scipy.stats.stats import pearsonr
import subprocess
from datetime import datetime


/tmp/ipykernel_2412853/1506328072.py:36: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


In [29]:
bash_dir= "bash"
output_dir = "output"
config_file = "zinc_gps_config.csv"

df_all = pd.read_csv(os.path.join(bash_dir, config_file),index_col=0) 
df_all = df_all.reset_index(drop=True)
df_all

,opt,beta1,beta2,seed,epoch,loss,val_mae,test_mae,lr,env,optimizer,learning_rate
0,AMSGrad,0.9,0.999,seed10,1,0.688913,0.513300,0.536601,0.0001,zinc,AMSGrad,0.00050
1,Lion,0.9,0.999,seed10,1,0.663569,0.523181,0.536426,0.0001,zinc,Lion,0.00025
2,Adam,0.9,0.999,seed10,1,0.692272,0.531897,0.551542,0.0001,zinc,Adam,0.00050
3,Adopt,0.9,0.999,seed10,1,0.712638,0.533868,0.563475,0.0001,zinc,Adopt,0.00050
4,Adam,0.9,0.900,seed10,1,0.697401,0.549562,0.583326,0.0001,zinc,Adam,0.00050
5,AdaGrad,0.9,0.999,seed10,1,0.667221,0.561272,0.582700,0.0010,zinc,AdaGrad,0.00250
6,Lamb,0.9,0.900,seed10,1,0.694632,0.563001,0.598583,0.0010,zinc,Lamb,0.00250
7,AdaBeliefW,0.9,0.999,seed10,1,0.683830,0.563764,0.603262,0.0001,zinc,AdaBeliefW,0.00050
8,AdaBelief,0.9,0.999,seed10,1,0.683830,0.563764,0.603262,0.0001,zinc,AdaBeliefW,0.00050
9,AdamPlus,0.9,0.900,seed10,1,0.727210,0.570630,0.588023,0.0001,zinc,AdamPlus,0.00050


In [45]:
df=df_all
# df=df_all.iloc[[2]]
# df=df_all.iloc[df_all.index != 6] #.iloc[[6]]
# df=df_all.iloc[df_all.index > 39] #.iloc[[6]]
# df=df_all.iloc[[0,1,2,3,4,5]]
# df.loc[df["opt"]=="Lion", "lr"]=0.0001
# df.loc[df["opt"]=="Adam41", "lr"]=0.0001
# df_config.loc[df_config["opt"]=="Adam4", "lr"]=0.001
df["lr"]=0.001
df

,opt,beta1,beta2,seed,epoch,loss,val_mae,test_mae,lr,env,optimizer,learning_rate
0,AMSGrad,0.9,0.999,seed10,1,0.688913,0.513300,0.536601,0.001,zinc,AMSGrad,0.00050
1,Lion,0.9,0.999,seed10,1,0.663569,0.523181,0.536426,0.001,zinc,Lion,0.00025
2,Adam,0.9,0.999,seed10,1,0.692272,0.531897,0.551542,0.001,zinc,Adam,0.00050
3,Adopt,0.9,0.999,seed10,1,0.712638,0.533868,0.563475,0.001,zinc,Adopt,0.00050
4,Adam,0.9,0.900,seed10,1,0.697401,0.549562,0.583326,0.001,zinc,Adam,0.00050
5,AdaGrad,0.9,0.999,seed10,1,0.667221,0.561272,0.582700,0.001,zinc,AdaGrad,0.00250
6,Lamb,0.9,0.900,seed10,1,0.694632,0.563001,0.598583,0.001,zinc,Lamb,0.00250
7,AdaBeliefW,0.9,0.999,seed10,1,0.683830,0.563764,0.603262,0.001,zinc,AdaBeliefW,0.00050
8,AdaBelief,0.9,0.999,seed10,1,0.683830,0.563764,0.603262,0.001,zinc,AdaBeliefW,0.00050
9,AdamPlus,0.9,0.900,seed10,1,0.727210,0.570630,0.588023,0.001,zinc,AdamPlus,0.00050


In [48]:
num_gpus = 2

env = "cifar10"
epochs = 200
seed = "52"
db=-30
decay_rate = 1
decay_steps = 1
batch_size = 100
weight_decay = 0.0005
scheduler = 0 # constant learning rate no scheduled decay
max_threads = 2

In [ ]:
running_procs = []

for i, row in df.iterrows():
    opt = row["opt"]
    beta1 = row["beta1"]
    beta2 = row["beta2"]
    lr = row["lr"]
    # lr = 0.0001
    
    gpu_id = i % num_gpus  # round-robin assignment
    env_copy = os.environ.copy()
    env_copy["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

    
    outname = f"{env}_epochs{epochs}_seed{seed}_{beta1}_{beta2}_{opt}_{lr}.txt"
    log_path = f"{output_dir}/{outname}"

    cmd = [
        "python3.10", "main_kuangliu.py",
        f"--seed={seed}",
        f"--learning_rate={lr}",
        f"--epochs={epochs}",
        f"--optimizer={opt}",
        f"--beta_1={beta1}",
        f"--beta_2={beta2}",
        f"--db={db}",
        f"--weight_decay={weight_decay}",
        f"--batch_size={batch_size}",
        f"--decay_steps={decay_steps}",
        f"--decay_rate={decay_rate}",
    ]

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}][Submitting]: {' '.join(cmd)} > {log_path}")

    with open(log_path, "w") as outfile:
        proc = subprocess.Popen(
            cmd, stdout=outfile, stderr=outfile, env=env_copy, 
            # preexec_fn=os.setsid,
        )
        running_procs.append(proc)

    # Limit number of concurrent jobs
    while len(running_procs) >= max_threads:
        for p in running_procs:
            if p.poll() is not None:  # finished
                running_procs.remove(p)
        time.sleep(30)  # wait a bit before checking again

# Wait for remaining jobs to finish
for p in running_procs:
    p.wait()


[2025-05-08 23:52:05][Submitting]: python3.10 main_kuangliu.py --seed=52 --learning_rate=0.001 --epochs=200 --optimizer=AMSGrad --beta_1=0.9 --beta_2=0.999 --db=-30 --weight_decay=0.0005 --batch_size=100 --decay_steps=1 --decay_rate=1 > output/cifar10_epochs200_seed52_0.9_0.999_AMSGrad_0.001.txt
[2025-05-08 23:52:05][Submitting]: python3.10 main_kuangliu.py --seed=52 --learning_rate=0.001 --epochs=200 --optimizer=Lion --beta_1=0.9 --beta_2=0.999 --db=-30 --weight_decay=0.0005 --batch_size=100 --decay_steps=1 --decay_rate=1 > output/cifar10_epochs200_seed52_0.9_0.999_Lion_0.001.txt
